###### Задание 1. Загрузка данных
Загрузите  файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate

In [2]:
# user id | item id | rating | timestamp
df_ratings = pd.read_csv('rating.csv', 
                        sep='\t', 
                        names=["user_id",
                        "item_id",
                        "rating",
                        "timestamp"],
                        dtype = {
                        "user_id": 'Int32', #01 
                        "item_id": 'Int64',  #02 
                        "rating":  'Float16',  
                        "timestamp": 'Int64', #03
                                 }
                        )

In [3]:
df_ratings.info()
df_ratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    100000 non-null  Int32  
 1   item_id    100000 non-null  Int64  
 2   rating     100000 non-null  float16
 3   timestamp  100000 non-null  Int64  
dtypes: Int32(1), Int64(2), float16(1)
memory usage: 2.4 MB


,user_id,item_id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [4]:
'''
movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western | 
              
'''              
df_movies = pd.read_csv('movies.csv', 
                        sep='|', 
                        names=["movie_id","movie_title", "release_date", "video_release_date",
                               "IMDb_URL", "unknown", "action", "adventure", "animation", 
                                "children_s", "comedy", "crime", "documentary", "drama", "fantasy",
                                "film-noir", "horror", "musical", "mystery", "romance", "sci-fi", 
                                "thriller", "war", "western"],
# https://stackoverflow.com/questions/5552555/unicodedecodeerror-invalid-continuation-byte                        
                        encoding='latin-1' 
                        )

In [5]:
df_movies.info()
df_movies.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            1682 non-null   int64  
 1   movie_title         1682 non-null   object 
 2   release_date        1681 non-null   object 
 3   video_release_date  0 non-null      float64
 4   IMDb_URL            1679 non-null   object 
 5   unknown             1682 non-null   int64  
 6   action              1682 non-null   int64  
 7   adventure           1682 non-null   int64  
 8   animation           1682 non-null   int64  
 9   children_s          1682 non-null   int64  
 10  comedy              1682 non-null   int64  
 11  crime               1682 non-null   int64  
 12  documentary         1682 non-null   int64  
 13  drama               1682 non-null   int64  
 14  fantasy             1682 non-null   int64  
 15  film-noir           1682 non-null   int64  
 16  horror

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,action,adventure,animation,children_s,...,fantasy,film-noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


###### Задание 2. 
Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок

In [6]:
df_groups = df_ratings.groupby('user_id').count()
df_groups.sort_values(by='rating', ascending=False).head(1)

,item_id,rating,timestamp
user_id,,,
405,737,737,737


Это пользователь с **user_id** равным **405**

###### Задание 3. 
Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь 

In [7]:
# Выберем фильмы оценённые пользователем 405:
df_405 = df_ratings.loc[df_ratings.user_id == 405]
df_405.head(5)

,user_id,item_id,rating,timestamp
12276,405,56,4.0,885544911
12383,405,592,1.0,885548670
12430,405,1582,1.0,885548670
12449,405,171,1.0,885549544
12460,405,580,1.0,885547447


Для построения модели нам нужны признаки. В качестве таковых будем использовать:
1. Год выхода
2. Жанры
3. Общее количество оценок
4. Суммарную оценку


###### Задание 4. 
Добавьте к датафрейму из задания 3 следующие столбцы:
1. По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет
2. Cтолбец с общим количеством оценок от всех пользователей на фильм
3. Cуммарной оценкой от всех пользователей


Чтобы добавить данные о жанрах нужно соединить датасеты **df_405** и   **df_movies** по столбцу идентификатора фильма:

In [8]:
df_row_join =  pd.merge(df_405, df_movies, how='inner', on=None, left_on='item_id', right_on='movie_id')
# Удалим лишние столбцы:
df_row_join.drop(columns=['timestamp','video_release_date','IMDb_URL','movie_title', 'movie_id'], inplace=True)
df_row_join.sort_values(by='item_id', ascending=True)

,user_id,item_id,rating,release_date,unknown,action,adventure,animation,children_s,comedy,...,fantasy,film-noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
239,405,2,1.0,01-Jan-1995,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
595,405,4,4.0,01-Jan-1995,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
362,405,5,4.0,01-Jan-1995,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
458,405,8,4.0,01-Jan-1995,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
23,405,11,4.0,01-Jan-1995,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,405,1588,1.0,21-Feb-1997,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
207,405,1589,1.0,23-May-1997,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
254,405,1590,1.0,06-Jun-1997,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
630,405,1591,1.0,21-Jan-1998,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Вычислим агрегированные значения по всем фильмам:
1. Cтолбец с общим количеством оценок от всех пользователей на фильм 
2. Cуммарной оценкой от всех пользователей

In [9]:
df_rating_group = df_ratings.groupby('item_id').agg({'rating':['sum','count']}).reset_index()
# Переименуем колонки:
df_rating_group.columns = ['item_id', 'rating_sum','rating_count']
df_rating_group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   item_id       1682 non-null   int64  
 1   rating_sum    1682 non-null   float16
 2   rating_count  1682 non-null   int64  
dtypes: float16(1), int64(2)
memory usage: 29.7 KB


Объединим датафреймы для построения модели:

In [10]:
df_for_model = pd.merge(df_row_join, df_rating_group, how='inner', on=None, left_on='item_id', right_on='item_id')
df_for_model.head(5)

,user_id,item_id,rating,release_date,unknown,action,adventure,animation,children_s,comedy,...,horror,musical,mystery,romance,sci-fi,thriller,war,western,rating_sum,rating_count
0,405,56,4.0,01-Jan-1994,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1600.0,394
1,405,592,1.0,01-Jan-1995,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,30.0,9
2,405,1582,1.0,01-Jan-1947,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,1
3,405,171,1.0,01-Jan-1991,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,252.0,65
4,405,580,1.0,01-Jan-1995,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,108.0,32


Преобразуем столбец **release_date** в формат date/time:

In [11]:
df_for_model['release_date'] = pd.to_datetime(df_for_model['release_date'])
df_for_model.info()
df_for_model.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 0 to 736
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   user_id       737 non-null    Int32         
 1   item_id       737 non-null    Int64         
 2   rating        737 non-null    float16       
 3   release_date  737 non-null    datetime64[ns]
 4   unknown       737 non-null    int64         
 5   action        737 non-null    int64         
 6   adventure     737 non-null    int64         
 7   animation     737 non-null    int64         
 8   children_s    737 non-null    int64         
 9   comedy        737 non-null    int64         
 10  crime         737 non-null    int64         
 11  documentary   737 non-null    int64         
 12  drama         737 non-null    int64         
 13  fantasy       737 non-null    int64         
 14  film-noir     737 non-null    int64         
 15  horror        737 non-null    int64     

,user_id,item_id,rating,release_date,unknown,action,adventure,animation,children_s,comedy,...,horror,musical,mystery,romance,sci-fi,thriller,war,western,rating_sum,rating_count
0,405,56,4.0,1994-01-01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1600.0,394
1,405,592,1.0,1995-01-01,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,30.0,9
2,405,1582,1.0,1947-01-01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,1
3,405,171,1.0,1991-01-01,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,252.0,65
4,405,580,1.0,1995-01-01,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,108.0,32


Выделим год выпуска фильма в отдельный столбец **release_year**:

In [12]:
df_for_model['release_year'] = df_for_model['release_date'].dt.year
df_for_model.info()
df_for_model.head(5).sort_values(by='rating_sum', ascending=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 0 to 736
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   user_id       737 non-null    Int32         
 1   item_id       737 non-null    Int64         
 2   rating        737 non-null    float16       
 3   release_date  737 non-null    datetime64[ns]
 4   unknown       737 non-null    int64         
 5   action        737 non-null    int64         
 6   adventure     737 non-null    int64         
 7   animation     737 non-null    int64         
 8   children_s    737 non-null    int64         
 9   comedy        737 non-null    int64         
 10  crime         737 non-null    int64         
 11  documentary   737 non-null    int64         
 12  drama         737 non-null    int64         
 13  fantasy       737 non-null    int64         
 14  film-noir     737 non-null    int64         
 15  horror        737 non-null    int64     

,user_id,item_id,rating,release_date,unknown,action,adventure,animation,children_s,comedy,...,musical,mystery,romance,sci-fi,thriller,war,western,rating_sum,rating_count,release_year
0,405,56,4.0,1994-01-01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1600.0,394,1994
3,405,171,1.0,1991-01-01,0,0,0,0,0,1,...,0,0,0,1,0,0,0,252.0,65,1991
4,405,580,1.0,1995-01-01,0,0,0,0,0,1,...,0,0,1,0,0,0,0,108.0,32,1995
1,405,592,1.0,1995-01-01,0,0,0,0,0,0,...,0,1,0,0,1,0,0,30.0,9,1995
2,405,1582,1.0,1947-01-01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,1,1947


**Теперь все готово и можно строить модель**

In [13]:
# опишем перечень колонок для X:
cols  =  ['unknown',
    'action',        
    'adventure',
 'animation',
 'children_s',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'film-noir',
 'horror',
 'musical',
 'mystery',
 'romance',           
 'sci-fi',
 'thriller',
 'war',
 'western',  
 'rating_sum',
 'rating_count',
 'release_year']

In [14]:
# Сформировать X и y из df_for_model, выбрав нужные колонки
X = df_for_model[cols]
y = df_for_model['rating']

In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 0 to 736
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   unknown       737 non-null    int64  
 1   action        737 non-null    int64  
 2   adventure     737 non-null    int64  
 3   animation     737 non-null    int64  
 4   children_s    737 non-null    int64  
 5   comedy        737 non-null    int64  
 6   crime         737 non-null    int64  
 7   documentary   737 non-null    int64  
 8   drama         737 non-null    int64  
 9   fantasy       737 non-null    int64  
 10  film-noir     737 non-null    int64  
 11  horror        737 non-null    int64  
 12  musical       737 non-null    int64  
 13  mystery       737 non-null    int64  
 14  romance       737 non-null    int64  
 15  sci-fi        737 non-null    int64  
 16  thriller      737 non-null    int64  
 17  war           737 non-null    int64  
 18  western       737 non-null    

Разделим данные на обучающую и тестовую выборки в пропорции 70% на 30%:

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

Проверим размерности выборок:

In [17]:
print(X_train.shape)
print(X_test.shape)

(515, 22)
(222, 22)


In [18]:
print(y_train.shape)
print(y_test.shape)

(515,)
(222,)


Обучим модель:

In [19]:
from  sklearn.linear_model import LinearRegression 
model =  LinearRegression() 
model.fit(X_train, y_train)  # метод обучается на данных и подбирает оптимальные коэффициенты

LinearRegression()

Посмотрим коэффициенты регрессионной модели:

In [20]:
model.coef_

array([ 0.        , -0.22687525, -0.31448619, -0.21060616,  0.43744157,
        0.20347135, -0.0193515 ,  0.51386454,  0.1528343 , -0.29330466,
       -0.42717305, -0.02484303,  0.26710488,  0.30787629, -0.12571284,
        0.08894853,  0.21079784, -0.36706937, -0.11316978, -0.00186138,
        0.01752502, -0.00643567])

и остаток:

In [21]:
model.intercept_

13.760996060252763

Оценим полученное качество модели:

In [22]:
y_train_pred = model.predict(X_train) 
y_test_pred = model.predict(X_test) 

Посмотрим как модель работает на тестовой выборке:

In [23]:
i = 0
for index_, y_test_val in y_test.items():
   
    print(f'{i+1:2d}. Реальное значение: {y_test_val:3.1f}  \
cмоделированное значение: {y_test_pred[i]:3.1f}  \
расхождение в {((y_test_pred[i]/y_test_val)-1)*100:3.1f} %')
    i += 1

 1. Реальное значение: 1.0  cмоделированное значение: 1.1  расхождение в 13.5 %
 2. Реальное значение: 1.0  cмоделированное значение: 1.8  расхождение в 77.4 %
 3. Реальное значение: 1.0  cмоделированное значение: 1.0  расхождение в -0.6 %
 4. Реальное значение: 1.0  cмоделированное значение: 1.3  расхождение в 30.5 %
 5. Реальное значение: 1.0  cмоделированное значение: 3.0  расхождение в 202.9 %
 6. Реальное значение: 1.0  cмоделированное значение: 3.1  расхождение в 205.7 %
 7. Реальное значение: 1.0  cмоделированное значение: 1.2  расхождение в 18.3 %
 8. Реальное значение: 1.0  cмоделированное значение: 2.0  расхождение в 103.0 %
 9. Реальное значение: 1.0  cмоделированное значение: 4.4  расхождение в 340.6 %
10. Реальное значение: 5.0  cмоделированное значение: 1.8  расхождение в -63.4 %
11. Реальное значение: 5.0  cмоделированное значение: 3.4  расхождение в -32.5 %
12. Реальное значение: 1.0  cмоделированное значение: 2.0  расхождение в 103.7 %
13. Реальное значение: 5.0  cмоде

In [24]:
# Оценить качество на тестовой выборке
from sklearn.metrics import mean_squared_error 
MSE_train = mean_squared_error(y_train, y_train_pred) 
MSE_test  = mean_squared_error(y_test,  y_test_pred) 

print(f'MSE тренировка:  {MSE_train:5.2f},  тестирование: {MSE_test:5.2f}') 

MSE тренировка:   1.09,  тестирование:  1.45


Коэффициент детерминации полученной модели:

In [25]:
model.score(X_test, y_test)

0.16438973767093068

In [26]:
from  sklearn.metrics  import r2_score 
R2_train =r2_score(y_train,  y_train_pred)
R2_test = r2_score(y_test, y_test_pred)
print(f'R2 тренировка: {R2_train:.3f},  тестирование: {R2_test:.3f}') 


R2 тренировка: 0.406,  тестирование: 0.164


**Вывод:** Для тестовой выборки модель показывет крайне низкий коэффициент детерминации, т.е. всего лишь 16% изменчивости зависимого выходного атрибута -  рейтинга  фильма может быть объяснено моделью, в то время как остальные 84% изменчивости все еще не учтены. Наблюдается переобучение модели, поскольку для тренировочной последовательности коэффициент детерминации выше, чем для тестовой модели.